In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset
import numpy as np
import os
from PIL import Image
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

In [2]:
plt.ion()

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(84),
        # transforms.CenterCrop(84),
        # 转换成tensor向量
        transforms.ToTensor(), # 对图像进行归一化操作
        # [0.485, 0.456, 0.406]，各通道的均值与标准差
        transforms.Normalize([0.5], [0.3])
    ]),
    'val': transforms.Compose([
        transforms.Resize(84),
       # transforms.CenterCrop(84),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.3])
    ]),
}

# 图像读入
def Load_Image_Information(path):
    # 图像存储路径
    image_Root_Dir= r'D:/download_By_Chome/data/acs_mosaic_2.0/train/for_model/'     # 所有标签为0，1的图像
    # 获取图像的路径
    iamge_Dir = os.path.join(image_Root_Dir, path)
    # 以L格式打开图像
    # Pytorch DataLoader就是使用PIL所读取的图像格式
    return Image.open(iamge_Dir).convert('L')


class my_Data_Set(nn.Module):
    def __init__(self, txt, transform=None, target_transform=None, loader=None):
        super(my_Data_Set, self).__init__()
        # 打开存储图像名与标签的txt文件
        fp = open(txt, 'r')
        # fp = open('D:/download_By_Chome/data/acs_mosaic_2.0/index/index.txt', 'r')
        images = []
        labels = []
        # 将图像名和图像标签对应存储起来
        for line in fp:
            line.strip('\n')
            line.rstrip()
            information = line.split()
            images.append(information[0]+' '+information[1])
            labels.append(int(information[2]))
        self.images = images
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    # 重写这个函数用来进行图像数据的读取
    def __getitem__(self, item):    # 实例对象通过[]运算符取值时，会调用它的方法__getitem__
        # 获取图像名和标签
        imageName = self.images[item]
        label = self.labels[item]
        # 读入图像信息
        image = self.loader(imageName)
        # 处理图像数据
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    # 重写函数，查看数据集中含有多少数据
    def __len__(self):
        return len(self.images)


In [3]:
# 生成Pytorch所需的DataLoader数据输入格式
train_dataset = my_Data_Set('D:/download_By_Chome/data/acs_mosaic_2.0/index/训练集.txt', transform=data_transforms['train'], loader=Load_Image_Information)
test_dataset = my_Data_Set('D:/download_By_Chome/data/acs_mosaic_2.0/index/测试集.txt', transform=data_transforms['val'], loader=Load_Image_Information)

In [ ]:
# test_dataset = my_Data_Set('D:/download_By_Chome/data/acs_mosaic_2.0/index/val.txt', transform=data_transforms['val'], loader=Load_Image_Information)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)   # DataLoader()数据加载器。结合数据集和采样器，并提供给定数据集的可迭代对象。
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

# 验证是否生成DataLoader格式数据
for data in train_loader:
    inputs, labels = data
    print(inputs)
    print(labels)
for data in test_loader:
    inputs, labels = data
    print(inputs)
    print(labels)

# 定义CNN
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1,20,5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(20, 16, 5)
        self.fc1 = nn.Linear(16 * 18 * 18, 800) # 最后一次卷积生成的图像矩阵的高度为16层
        self.fc2 = nn.Linear(800, 120)
        self.fc3 = nn.Linear(120, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 18 * 18)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

net = Net()

#训练，n=220
cirterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.5)
for epoch in range(220):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)
        optimizer.zero_grad()                        # 优化器清零
        outputs = net(inputs)
        loss = cirterion(outputs, labels)
        loss.backward()
        optimizer.step()                         #优化
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('finished training!')

tensor([[[[ 0.9608,  1.6013,  1.4575,  ...,  1.3660,  1.4575,  1.4837],
          [ 1.0523,  1.4706,  1.5229,  ...,  1.5098,  0.9869,  0.5033],
          [ 0.6993,  1.2876,  1.3268,  ...,  1.2876,  1.1569,  0.6863],
          ...,
          [ 0.8824,  1.4444,  1.5098,  ...,  0.7908,  1.4444,  1.4837],
          [ 1.6275,  1.6144,  1.4837,  ...,  1.3922,  1.3007,  1.3791],
          [ 1.6275,  1.5229,  1.2484,  ...,  1.5752,  1.3007,  1.3007]]],


        [[[ 1.6275,  1.6405,  1.6405,  ...,  1.6536,  1.5882,  1.5098],
          [ 1.5882,  1.5882,  1.5882,  ...,  1.6405,  1.6013,  1.6275],
          [ 1.5882,  1.4967,  1.6013,  ...,  1.6536,  1.5621,  1.6144],
          ...,
          [ 1.6536,  1.6405,  1.6144,  ...,  1.6405,  1.6144,  1.5359],
          [ 1.6275,  1.6667,  1.6536,  ...,  1.4314,  1.5098,  1.6275],
          [ 1.6405,  1.6536,  1.6667,  ...,  1.5229,  1.5752,  1.6405]]],


        [[[ 0.5425, -0.4902, -0.2157,  ...,  1.5621,  1.2745,  1.1176],
          [ 0.4902,  0.764

In [ ]:
#测试
correct = 0
total = 0
with torch.no_grad():
    '''禁用梯度计算。当不会调用 Tensor.backward() 时，减少原本需要 requires_grad=True的计算的内存消耗。在这种模式下，每次计算的结果都会有requires_grad=False 使输入有requires_grad=True 。这个上下文管理器是线程本地的；它不会影响其他线程中的计算。也起到装饰器的作用。
    '''
    for data in test_loader:    # test_loader = DataLoader(test_dataset, batch_size=10, shuffle=True)

        images, labels = data
        outputs = net(Variable(images))
        '''
在PyTorch0.4.0之后Variable 已经被PyTroch弃用
Variable不再是张量使用autograd的必要条件
只需要将张量的requires_grad设为True该张量就会自动支持autograd运算
在新版的PyTorch中Variable(tensor)和Varialbe(tensor, requires_grad)还能继续使用，但是返回的是tensor变量，而不是Variable变量
使用Variable.data的效果和使用tensor.data一样
诸如Variable.backward()，Variable.detach()，Varialbe.register_hook()等方法直接移植到了tensor上，在tensor上可以直接使用同名的函数
'''

        _, predicted = torch.max(outputs.data, dim=1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

print('Accuracy of the network on test images: %d %%' % (100 * correct / total))

In [ ]:
# 图像读入
class Data_Set(nn.Module):
    def __init__(self, txt, transform=None, target_transform=None, loader=None):
        super(Data_Set, self).__init__()
        # 打开存储图像名与标签的txt文件
        fp = open('D:/download_By_Chome/data/acs_mosaic_2.0/index/obj.txt', 'r')
        images = []
        labels = []
        # 将图像名和图像标签对应存储起来
        for line in fp:
            line.strip('\n')
            line.rstrip()
            information = line.split()
            images.append(information[0])
            labels.append(int(information[1]))
        self.images = images
        self.labels = labels
        self.transform = transform
        self.target_transform = target_transform
        self.loader = loader

    # 重写这个函数用来进行图像数据的读取
    def __getitem__(self, item):    # 实例对象通过[]运算符取值时，会调用它的方法__getitem__
        # 获取图像名和标签
        imageName = self.images[item]
        label = self.labels[item]
        # 读入图像信息
        image = self.loader(imageName)
        # 处理图像数据
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    # 重写函数，查看数据集中含有多少数据
    def __len__(self):
        return len(self.images)

'''

'''

def Load_Image(path):
    # 图像存储路径
    image_Dir= r'D:/download_By_Chome/data/acs_mosaic_2.0/JPGS'     # 所有图像
    # 获取图像的路径
    iamge_Dir = os.path.join(image_Dir, path)
    # 以L格式打开图像
    # Pytorch DataLoader就是使用PIL所读取的图像格式
    return Image.open(iamge_Dir).convert('L')

# 目标数据
obj_dataset = Data_Set('D:/download_By_Chome/data/acs_mosaic_2.0/index/obj.txt', transform=data_transforms['val'], loader=Load_Image)
obj_loader = DataLoader(obj_dataset, batch_size=10, shuffle=True)

In [ ]:
prediction=[]
with torch.no_grad():
    for data in obj_loader:
        images, labels = data
        outputs = net(Variable(images))
        __, predicted = torch.max(outputs.data, dim=1)
        # total += labels.size(0)
        # correct += (predicted == labels).sum()
        prediction.append(predicted)
print(len(prediction))

In [ ]:
pre=[]
for i in range(len(prediction)):    # len(prediction)=2095
    for j in range(len(prediction[i])):
        pre.append(int(prediction[i][j]))

In [ ]:
count=0
for i in range(len(pre)):
    if pre[i]==1:
        count+=1
print(count/len(pre))

In [ ]:
lis=os.listdir('''D:/download_By_Chome/data/acs_mosaic_2.0/JPGS''')
c=[]
for i in range(len(pre)):
    if pre[i]>0:
        c.append(lis[i])

In [ ]:
fig = plt.figure(figsize=(12,8),dpi=80)
for i in range(1,13):
    ax=fig.add_subplot(3,4,i)
    lo=c[np.random.randint(1,len(c))]
    img_path ="D:/download_By_Chome/data/acs_mosaic_2.0/JPGS/"
    image=plt.imread(img_path+lo)
    ax.imshow(image,cmap='plasma')
    s=lo.split('_',)
    ax.set_title('{}\n{}'.format(s[1],s[2]))
    ax.axis('off')

plt.show()

import matplotlib.pyplot as plt

img = plt.imread(img_path)
fig = plt.figure('show picture')
plt.imshow(img)
plt.show()

In [ ]:
prediction